In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [3]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the token
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

Token loaded: hf_S...HLUg


In [5]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

/Users/helenauta.bolt/Documents/GitHub/policyengine-uk/policyengine_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Level 1: 20 GBP pw uplift to child element of UC
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant("2026-01-01"), stop=instant("2031-12-31"), value=292.81+80.00
    )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant("2026-01-01"), stop=instant("2031-12-31"), value=339.00+80.00
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)




In [ ]:
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calculate("gov_spending", 2026).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in 2026")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calculate("gov_spending", 2029).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in 2029" )

net_income_baseline = baseline.calculate("household_net_income", 2026)
net_income_reformed = reformed.calculate("household_net_income", 2026)
net_income_baseline_29 = baseline.calculate("household_net_income", 2029)
net_income_reformed_29 = reformed.calculate("household_net_income", 2029)
total_net_income_baseline = baseline.calculate("household_net_income", 2029).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2029).sum()
diff=total_net_income_reformed-total_net_income_baseline
print(f"Total net income in 2029: {diff/1e9:.2f}bn")
print(f"Baseline spending 2026: £{net_income_baseline.sum() / 1e9:.2f}bn")
print(f"Reformed spending 2026: £{net_income_reformed.sum()  / 1e9:.2f}bn")
print(f"Baseline spending 2029: £{net_income_baseline_29.sum()  / 1e9:.2f}bn")
print(f"Reformed spending 2029: £{net_income_reformed_29.sum()  / 1e9:.2f}bn")


In [ ]:
uc_variable = "universal_credit"  # 

uc_baseline_2026 = baseline.calculate(uc_variable, 2026).sum()
uc_reformed_2026 = reformed.calculate(uc_variable, 2026).sum()
uc_diff_2026 = uc_reformed_2026 - uc_baseline_2026

uc_baseline_2029 = baseline.calculate(uc_variable, 2029).sum()
uc_reformed_2029 = reformed.calculate(uc_variable, 2029).sum()
uc_diff_2029 = uc_reformed_2029 - uc_baseline_2029
print(f"Universal Credit spending in 2026: Baseline: £{uc_baseline_2026 / 1e9:.2f}bn, Reformed: £{uc_reformed_2026 / 1e9:.2f}bn, Difference: £{uc_diff_2026 / 1e9:.2f}bn")
print(f"Universal Credit spending in 2029: Baseline: £{uc_baseline_2029 / 1e9:.2f}bn, Reformed: £{uc_reformed_2029/ 1e9:.2f}bn, Difference: £{uc_diff_2029 / 1e9:.2f}bn")

In [ ]:
diff = net_income_reformed*net_income_reformed.weights - net_income_baseline*net_income_baseline.weights
gainers = (diff > 0).sum()
print(gainers)
total=net_income_reformed.sum()-net_income_baseline.sum()
print(f"Avg gain of gainers 2026: {total/gainers}")

In [ ]:
diff = net_income_reformed_29*net_income_reformed_29.weights - net_income_baseline_29*net_income_baseline_29.weights
gainers = (diff > 0).sum()
print(gainers)
total=net_income_reformed_29.sum()-net_income_baseline_29.sum()
print(f"Avg gain of gainers 2029: {total/gainers}")

In [ ]:
all_variables = baseline.tax_benefit_system.variables

# Print the number of variables available
print(f"Total number of variables: {len(all_variables)}")
print("Sample variables:")
for i, var_name in enumerate(list(all_variables.keys())[:100]):
    print(f"  - {var_name}")

In [21]:
# Step 1: Make sure to include the weights column in your variables
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "household_weight", "person_weight"]  # Include the weight variable
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, period=2032, use_weights=True).astype(float)
weighted_persons= baseline_person_df["person_weight"].sum()

# Step 2: Create a household-level dataframe by dropping duplicate household IDs
household_df = baseline_person_df.drop_duplicates('household_id')

# Step 3: Calculate the weighted number of households
# Sum the weights column, not the entire dataframe
weighted_households = household_df["household_weight"].sum()

print(f"Total weighted number of households: {weighted_households:,.0f}")
print(f"Total weighted number of people: {weighted_persons:,.0f}")

Total weighted number of households: 35,238,752
Total weighted number of people: 76,174,602


In [ ]:
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026,use_weights=True).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026,use_weights=True).astype(float)
# baseline_person_df["weight"] = net_income_baseline.weights
# household_df = baseline_person_df.drop_duplicates("household_id")[["household_id", "weight"]]
# Calculate poverty rate before reform
n_households=baseline.calculate("people",2025).benunits.count()
print(f"Number of households in the simulation: {n_households}")
collapsed_baseline = baseline_person_df.groupby("household_id").agg(in_poverty=("in_poverty", "first"))
poverty_before = collapsed_baseline["in_poverty"].mean()  # Use mean() instead of sum()/len

# Calculate poverty rate after reform
collapsed_reform = reformed_person_df.groupby("household_id").agg(in_poverty=("in_poverty", "first"))
poverty_after = collapsed_reform["in_poverty"].mean()  # Use mean() instead of sum()/len
diff_poverty=poverty_before-poverty_after
print(f"Total poverty difference: {diff_poverty}")
print(f"%in poverty: {poverty_before}")
print(f"% in poverty after: {poverty_after}")
print(f"Total reduction of HH in poverty: {diff_poverty*28400000}")

In [ ]:
#Level2:  Work allowance increased to 1000 GBP  pa
def change_tax_parameters(parameters):
    # print(type(parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing))
    # latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing["values"].get("2025-04-01")
    april_2025_date = "2025-04-01"
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing(april_2025_date)
    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant("2026-01-01"), value=latest_value+83.33
    )
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing(april_2025_date)
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant("2026-01-01"), value=latest_value+83.33
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
print(f"Cost 2026: £{round(revenue / 1e+9, 1)}bn")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calc("gov_spending", 2029).sum()
print(f"Cost 2029: £{round(revenue / 1e+9, 1)}bn")

In [ ]:
#Level 2: Change child limit
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant("2026-01-01"), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calc("gov_spending", 2029).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")

##RUN 2 CHILD LIMIT REFORM: 

In [ ]:
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
difference_person_df = reformed_person_df - baseline_person_df
total_net_income_baseline = baseline.calculate("household_net_income", 2026).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2026).sum()

net_cost = total_net_income_reformed - total_net_income_baseline

print(f"This reform would cost £{net_cost / 1e9:.1f}bn")

In [ ]:

def modify_parameters(parameters):
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_OLD.update(start=instant("2024-01-01"), value=738.82)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_YOUNG.update(start=instant("2024-01-01"), value=618.51)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_OLD.update(start=instant("2024-01-01"), value=528.74)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_YOUNG.update(start=instant("2024-01-01"), value=452.11)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)

baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2025).sum() - baseline.calc("gov_spending", 2025).sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"
